### Implemeting character-level LSTM text generation

In [49]:
#import keras
import numpy as np
from keras.utils.data_utils import get_file

In [50]:
path = get_file('nietzsche', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length: ', len(text))

Corpus length:  600901


In [51]:
text[0:100]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all ph'

#### Vectorising sequence of characters

In [52]:
maxlen = 60 # extract 60 chars of sequence
step = 3 # Sample a new sequence every 3 characters
sentence =[] # Features 
next_char = [] #Target

In [53]:
count = 3
for i in range(0, len(text)-maxlen, step):
    if count == 0:
        break
    else:
        print(text[i : i+maxlen])
        print(text[i+maxlen])
        count -= 1
    

preface


supposing that truth is a woman--what then? is the
r
face


supposing that truth is a woman--what then? is there 
n
e


supposing that truth is a woman--what then? is there not
 


In [54]:
for i in range(0, len(text)-maxlen, step):
    sentence.append(text[i : i+maxlen])
    next_char.append(text[i+maxlen])
print(f"Number of sequence: {len(sentence)}")

Number of sequence: 200281


In [55]:
# List of unique characters in the corpus. We need this for generating proabability distribution of next token
chars = sorted(list(set(text)))
print(f"Unique characters: {len(chars)}")

Unique characters: 59


In [56]:
chars[1:5]

[' ', '!', '"', "'"]

In [57]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [58]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 '¤': 53,
 '¦': 54,
 '©': 55,
 '«': 56,
 'ã': 57,
 '†': 58}

In [59]:
print('vectorization...')
x = np.zeros((len(sentence), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentence), len(chars)), dtype=np.bool)

vectorization...


In [60]:
x.shape, y.shape

((200281, 60, 59), (200281, 59))

In [61]:
#For Testing
for i, sentnc, in enumerate(sentence):
    print(f"i : {i}")
    print(f"sentnc : {sentnc}")
    for t, char in enumerate(sentnc):
        print(f"t : {t}")
        print(f"chars : {char}")
    break

i : 0
sentnc : preface


supposing that truth is a woman--what then? is the
t : 0
chars : p
t : 1
chars : r
t : 2
chars : e
t : 3
chars : f
t : 4
chars : a
t : 5
chars : c
t : 6
chars : e
t : 7
chars : 

t : 8
chars : 

t : 9
chars : 

t : 10
chars : s
t : 11
chars : u
t : 12
chars : p
t : 13
chars : p
t : 14
chars : o
t : 15
chars : s
t : 16
chars : i
t : 17
chars : n
t : 18
chars : g
t : 19
chars :  
t : 20
chars : t
t : 21
chars : h
t : 22
chars : a
t : 23
chars : t
t : 24
chars :  
t : 25
chars : t
t : 26
chars : r
t : 27
chars : u
t : 28
chars : t
t : 29
chars : h
t : 30
chars :  
t : 31
chars : i
t : 32
chars : s
t : 33
chars :  
t : 34
chars : a
t : 35
chars :  
t : 36
chars : w
t : 37
chars : o
t : 38
chars : m
t : 39
chars : a
t : 40
chars : n
t : 41
chars : -
t : 42
chars : -
t : 43
chars : w
t : 44
chars : h
t : 45
chars : a
t : 46
chars : t
t : 47
chars :  
t : 48
chars : t
t : 49
chars : h
t : 50
chars : e
t : 51
chars : n
t : 52
chars : ?
t : 53
chars :  
t : 54
chars : i

In [62]:
for i, sentnc, in enumerate(sentence):
    for t, char in enumerate(sentnc):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_char[i]]] = 1

#### Single layer LSTM model for next character prediction

In [63]:
from keras import layers
from keras.models import Sequential

model = Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#### Function to sample the next characer

In [64]:
def sample(preds, temperture=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperture
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#### Text generation loop

In [66]:
import random, sys
# Will train for 60 epochs
# Each epoch select a text sequence from random index
# Then generate 400 characters for each temperture value
for epoch in range(1, 60):
    print(f"epoch : {epoch}")
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    # Will select text at random index
    generated_text = text[start_index: start_index + maxlen]
    print(f"--- Generating with seed: {generated_text} ")
    
    for temperture in [0.2, 0.5, 0.7, 1.0, 1.2]:
        print(f"----- temperture: {temperture}")
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperture)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)

epoch : 1
1565/1565 [==============================] - 176s 112ms/step - loss: 2.1173
--- Generating with seed: ed for,
protected, and indulged, like some delicate, strange 
----- temperture: 0.2
ed for,
protected, and indulged, like some delicate, strange the soment of the self and the preation and the fore the prestion of the self and the sence the sore the some the soples and the some the beat of the self the seching the prothing the self the preation of the propere and the mant of the some the proust of the soul the sould the inters of the proper and the preation of the sence of the reation the mast of the soman the greation of the sould and th----- temperture: 0.5
ation the mast of the soman the greation of the sould and the beres and is and mant soud to crened thith it as in excacie of the sond the gance in the vertoud the hese the pation of the merelt there spelition, at and and the sting and and dease the mone the ast and canmer and depesthing of al the presore to the serition 

in spite of their regulative impersons and in the sperine sporition and the spirit and and interest and more the spirit and sporition of the world in the spirit and the fart of the fart of the stand and the sporition of the sperion of the sperience of the hast of the states of the world and still in the condining the really for the stand of the fart of the stand in the spirit and sperience of the same the condering the sand to ----- temperture: 0.5
 spirit and sperience of the same the condering the sand to be the reade of him has a sting this in theis and we lenged
in the every the reaply in the defing and and in which the being the act of the extrom the mode to the consect and master and so the indeptions, he wast more to be all the then who we all the opposent of more concimation of the expecting the sense the sting and the while and the spose of the spone, and speriness of the desention of the co----- temperture: 0.7
spose of the spone, and speriness of the desention of the concern

an in the conscience of the prisent of the faing of the speristor of the thing of the same there is the spirit of the action of the stall sense intermance the more the complession of the hand of the fatule that some not to under the sabord of the faish of any perhaps in mysing the spenist of an interthers of the domant of the fartly of the will has been even the farm of the provess of the profe-sting man in the bearder and pation of contine, the himself se----- temperture: 0.7
ing man in the bearder and pation of contine, the himself sempents of morality only scholetifules to as hand comal vongutering in in
the stact to certies of men importing to gut has ever there-denishing that at a peaicial of the fallo its something and and that it is the recossion of only to very ternion of the than the world, there starnable injurger the hels of the should in the endiention of the philosopher, who eart their nemelicivation of the one the ----- temperture: 1.0
e philosopher, who eart their nemeli

same to the person of the soul of the fair to the sumpers of philosophy instincts of the fair of the world become a god of a man as a most perease to the experience of the sumply as the will has been a past of which reams and something the porter of mirally of philosophy, the sumpresents of the feeling of the condition of the point of the probless of a men to the here all that it is not perhaps a god of sumple to the earound of a with perhaps of the compul----- temperture: 0.7
god of sumple to the earound of a with perhaps of the compulate of himself withor as the really morality; inderable that the came of dearned, and philosophy also a such a rearly of the accestary are all the endically and actrood conscious as the commank of all hels. the hemely of the hastiness of human, that the every of the fauther of wimally against
the myst actinon and more
god and surcement to the "sumpter, a higher in the rest of spirit very perpoutio----- temperture: 1.0
 the "sumpter, a higher in the rest 

he spirit and moral the standard of the spirit of the spirit and special life it was not the intertions, regards the world of all to be a condition of the spirit which he is a subject distingures of the sense things the same new as act of a thought the experience with the uncartages of morality, one its which the perist condicion of the perile most perhaps in the condition of string himself has himself is no most itself which the spirit conditions and man ----- temperture: 0.7
mself is no most itself which the spirit conditions and man for a thing" and pervaises the power things of his indefing"
and attented the samateraurs of "from the faires for order
unlefty. the noble to be sticler there are even as the conducated please the grast with the prive of order to another should refersed and spirit and consible as a way prive with the believe and nobitity one for the wordos in the had to himself, it as with a conduct demandity of t----- temperture: 1.0
s in the had to himself, it as with 

...

supposed assumpted of the super-greak to the manner a peosles only when the----- temperture: 0.7
ted of the super-greak to the manner a peosles only when the kind of life
become strong and time is saight
givence its worther," when
the feeling the act of fact that staris, let us slee which the "modern its swapting than almost pread and carry and something which we enchand conoraces of interpretation of the other. new the misureditate men mistrian, and nothing that
still his sonceaticiansiagion and rid who believe however, to be strengthion-lores at
als----- temperture: 1.0
 and rid who believe however, to be strengthion-lores at
als--and
also even and danger, and sinking make oper any life, expressed including it matters for
a-stangar4 as "tations
and smolly welf "miracs," a stepping and comes
he hitherto below you doary, (to the same timing in the
humaniesty of man is suffered untards
the best shoptephains free "namil," with presend, and difficult frued mirtelmant people conduct as in 

entarmed. one mushys for anor nature should is sadd. as let up a short of men of living taste of the end of the attemponelled. "but it
perhaps some time of the subjects human and men of protists, and cally----- temperture: 1.0
me time of the subjects human and men of protists, and cally in order that a sablefticiest
depth for those much
and nob-driarc, and adss of his problum, by the
roce of delight is then choracter even and cogrising cal longsorian fragnilically and
command--famal pain: the sear most a
conclusion. "i my hand good namely, he has adornd special prilokors; and can be we an superting, in
the purposent, firstly,
creations are them "dangering, the mijure is the
rena----- temperture: 1.2
rstly,
creations are them "dangering, the mijure is the
renait newass and mimulan devierantly richer causeful, sympt to pasce
corrisplened my agesari, to
as. in virit.
sup for some sinwilvor. for rempinual love live-griengrur, bad britual diselfings? ot it it dodscahually is evil
proced as 

to idea of whatever seemed (tratifulm late it which call them as imserts every existence and science of nom-life contraditide, for the fastent to and sensation. the saint for the comprehensive may har don the precite of sin not
messumple that cannot and sut handfers the underance whic well coudle amstrady, it is the day of world.=--to democration and accientl himself was is not somethi----- temperture: 1.2
ld.=--to democration and accientl himself was is not something, always soulthirgoe;--this
rame therein, and workd!
prodice ays "morals,"
lessonme,,
--; 1k. ke ma
porvars ide doem in his
sxisuation or,
beyonds, and there eurates--and
not be moges your indisposited: whenceg is refear indistance.--nought
and sove opposs runes
for trmay,
as a stumber) that necimeds to for betires lover
than edreynce. and bleasves to been luf--inffurstfsion of the vaightor of "epoch : 51
1565/1565 [==============================] - 189s 121ms/step - loss: 1.1087
--- Generating with seed: to whom this pove

of superflices it: it natist,
no othirig,non, and
intellectual, sympathy dame
they as else,
him, in impulates to like that art what stially dessists and inscentoned time himself of selfing, in all that estrund take must. be
oxtom not the word no douss
 fannally free from costixion, he ceets of
our can the kind which germanw strength.

veil--from the sthemely,
rase shall, by, has he dissestrated of lightsious
grout as expreepoch : 54
1565/1565 [==============================] - 177s 113ms/step - loss: 1.1003
--- Generating with seed: to the remotest ages and indicate the path that would be tak 
----- temperture: 0.2
to the remotest ages and indicate the path that would be taken to the same attempt to the same time the other more spirit and every farse of indeed, he is not unway to the superiority of the fact that is the power of such certain conceptions, and he can be a science of the commonplece with the same as the sentiments of the part of the same as the said of the problem for the 

1565/1565 [==============================] - 183s 117ms/step - loss: 1.0924
--- Generating with seed: , ye free spirits!
these are the heavy distant thoughts and  
----- temperture: 0.2
, ye free spirits!
these are the heavy distant thoughts and sense of individuals, to dety of a manner and deterile of the fact that is all discidlences of the same distrauses and strong and decessive is the world of the moral complainting is a some of the same disincer, as in the held and more a mediocretous great themselves and the exception of the same distinction of the free spirit in the same deneining of the same different of the same distrausenes to ----- temperture: 0.5
deneining of the same different of the same distrausenes to a prood his object, connections of the consequences of the free full of the such cultured to medion of the conscience is not to only a high renderounts of the sightence of the case of the will, of whom untelfarse and strong oness of--pirtually and society, to a long-diffe